In [1]:
!gdown --fuzzy "https://drive.google.com/uc?id=1KycQj4dik91RuBGvbhDJou7YDQEKAH2Z" -O MSFD.zip

Downloading...
From (original): https://drive.google.com/uc?id=1KycQj4dik91RuBGvbhDJou7YDQEKAH2Z
From (redirected): https://drive.google.com/uc?id=1KycQj4dik91RuBGvbhDJou7YDQEKAH2Z&confirm=t&uuid=09a9a825-a38c-402b-bcee-f79d7b66ee99
To: /kaggle/working/MSFD.zip
100%|███████████████████████████████████████| 2.25G/2.25G [00:15<00:00, 143MB/s]


In [ ]:
!unzip MSFD.zip

Archive:  MSFD.zip
   creating: MSFD/
   creating: MSFD/1/
   creating: MSFD/1/face_crop/
  inflating: MSFD/1/face_crop/000000_1.jpg  
  inflating: MSFD/1/face_crop/000001_1.jpg  
  inflating: MSFD/1/face_crop/000002_1.jpg  
  inflating: MSFD/1/face_crop/000003_1.jpg  
  inflating: MSFD/1/face_crop/000004_1.jpg  
  inflating: MSFD/1/face_crop/000005_1.jpg  
  inflating: MSFD/1/face_crop/000006_1.jpg  
  inflating: MSFD/1/face_crop/000006_2.jpg  
  inflating: MSFD/1/face_crop/000007_1.jpg  
  inflating: MSFD/1/face_crop/000007_2.jpg  
  inflating: MSFD/1/face_crop/000008_1.jpg  
  inflating: MSFD/1/face_crop/000009_1.jpg  
  inflating: MSFD/1/face_crop/000010_1.jpg  
  inflating: MSFD/1/face_crop/000011_1.jpg  
  inflating: MSFD/1/face_crop/000011_2.jpg  
  inflating: MSFD/1/face_crop/000011_3.jpg  
  inflating: MSFD/1/face_crop/000011_4.jpg  
  inflating: MSFD/1/face_crop/000011_5.jpg  
  inflating: MSFD/1/face_crop/000011_6.jpg  
  inflating: MSFD/1/face_crop/000012_1.jpg  
  inflatin

In [1]:
import tensorflow as tf
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split

class MSFDDataset(tf.keras.utils.Sequence):
    def __init__(self, image_files, segmentation_files, img_size=(128, 128), batch_size=32):
        self.image_files = image_files
        self.segmentation_files = segmentation_files
        self.img_size = img_size
        self.batch_size = batch_size

    def __len__(self):
        return int(np.ceil(len(self.image_files) / self.batch_size))

    def __getitem__(self, idx):
        batch_images = self.image_files[idx * self.batch_size : (idx + 1) * self.batch_size]
        batch_segmentations = self.segmentation_files[idx * self.batch_size : (idx + 1) * self.batch_size]
        
        images = []
        segmentations = []
        
        for img_path, seg_path in zip(batch_images, batch_segmentations):
            image = cv2.imread(img_path)
            if image is None:
                continue  
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            image = cv2.resize(image, (self.img_size[1], self.img_size[0])).astype(np.float32) / 255.0
            
            segmentation = cv2.imread(seg_path, cv2.IMREAD_GRAYSCALE)
            if segmentation is None:
                continue  
            segmentation = cv2.resize(segmentation, (self.img_size[1], self.img_size[0]), interpolation=cv2.INTER_NEAREST)
            segmentation = segmentation.astype(np.uint8)  
            
            images.append(image)
            segmentations.append(segmentation)
        
        images = np.array(images, dtype=np.float32)
        segmentations = np.array(segmentations, dtype=np.uint8)
        
        return images, segmentations

# Load dataset paths
dataset_path = "/kaggle/working/MSFD/1"
face_dir = os.path.join(dataset_path, "face_crop")
segmentation_dir = os.path.join(dataset_path, "face_crop_segmentation")

image_files = []
segmentation_files = []

for img_name in os.listdir(face_dir):
    img_path = os.path.join(face_dir, img_name)
    seg_path = os.path.join(segmentation_dir, img_name)
    
    if os.path.exists(img_path) and os.path.exists(seg_path):
        image_files.append(img_path)
        segmentation_files.append(seg_path)

# Train-test split (80-20)
train_images, val_images, train_masks, val_masks = train_test_split(image_files, segmentation_files, test_size=0.2, random_state=42)

# Create dataset instances
train_dataset = MSFDDataset(train_images, train_masks)
val_dataset = MSFDDataset(val_images, val_masks)

print(f"Training samples: {len(train_images)}, Validation samples: {len(val_images)}")

Training samples: 7505, Validation samples: 1877


In [3]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

# Convert dataset to NumPy arrays
def get_data_from_generator(generator):
    X, Y = [], []
    for i in range(len(generator)):
        x_batch, y_batch = generator[i]
        X.append(x_batch)
        Y.append(y_batch)
    return np.concatenate(X), np.concatenate(Y)

# Load datasets
X_train, Y_train = get_data_from_generator(train_dataset)
X_val, Y_val = get_data_from_generator(val_dataset)

# Ensure masks are in the correct format
Y_train = Y_train[..., np.newaxis] / 255.0  # Normalize mask
Y_val = Y_val[..., np.newaxis] / 255.0

# IoU and Dice Metrics
def iou_metric(y_true, y_pred):
    y_pred = tf.cast(y_pred > 0.5, tf.float32)
    intersection = tf.reduce_sum(y_true * y_pred)
    union = tf.reduce_sum(y_true) + tf.reduce_sum(y_pred) - intersection
    return intersection / (union + 1e-7)

def dice_metric(y_true, y_pred):
    y_pred = tf.cast(y_pred > 0.5, tf.float32)
    intersection = tf.reduce_sum(y_true * y_pred)
    return (2. * intersection) / (tf.reduce_sum(y_true) + tf.reduce_sum(y_pred) + 1e-7)

# U-Net Model Definition
def build_unet():
    input_shape = (128, 128, 3)
    dropout_rate = 0.3
    use_batchnorm = True
    base_filters = 32
    depth = 3

    inputs = tf.keras.layers.Input(input_shape)
    x = inputs
    skips = []

    # Encoder
    for i in range(depth):
        filters = base_filters * (2**i)
        x = tf.keras.layers.Conv2D(filters, (3, 3), activation='relu', padding='same')(x)
        if use_batchnorm:
            x = tf.keras.layers.BatchNormalization()(x)
        x = tf.keras.layers.Conv2D(filters, (3, 3), activation='relu', padding='same')(x)
        if use_batchnorm:
            x = tf.keras.layers.BatchNormalization()(x)
        skips.append(x)
        x = tf.keras.layers.MaxPooling2D((2, 2))(x)
        x = tf.keras.layers.Dropout(dropout_rate)(x)

    # Bottleneck
    filters = base_filters * (2**depth)
    x = tf.keras.layers.Conv2D(filters, (3, 3), activation='relu', padding='same')(x)
    if use_batchnorm:
        x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Conv2D(filters, (3, 3), activation='relu', padding='same')(x)
    if use_batchnorm:
        x = tf.keras.layers.BatchNormalization()(x)

    # Decoder
    for i in reversed(range(depth)):
        filters = base_filters * (2**i)
        x = tf.keras.layers.UpSampling2D((2, 2))(x)
        x = tf.keras.layers.Concatenate()([x, skips[i]])
        x = tf.keras.layers.Conv2D(filters, (3, 3), activation='relu', padding='same')(x)
        if use_batchnorm:
            x = tf.keras.layers.BatchNormalization()(x)
        x = tf.keras.layers.Conv2D(filters, (3, 3), activation='relu', padding='same')(x)
        if use_batchnorm:
            x = tf.keras.layers.BatchNormalization()(x)
        x = tf.keras.layers.Dropout(dropout_rate)(x)

    outputs = tf.keras.layers.Conv2D(1, (1, 1), activation='sigmoid')(x)

    model = tf.keras.Model(inputs, outputs)
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
        loss="binary_crossentropy",
        metrics=["accuracy", iou_metric, dice_metric]
    )
    return model

# Build and train the model
model = build_unet()
history = model.fit(
    X_train, Y_train,
    validation_data=(X_val, Y_val),
    batch_size=16,
    epochs=50,
    callbacks=[
        EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True),
        ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=3, verbose=1)
    ]
)

# Save best model
model.save("unet_best.h5")


Epoch 1/50
470/470 ━━━━━━━━━━━━━━━━━━━━ 83s 119ms/step - accuracy: 0.5830 - dice_metric: 0.8063 - iou_metric: 0.6883 - loss: 0.3160 - val_accuracy: 0.6193 - val_dice_metric: 0.9165 - val_iou_metric: 0.8467 - val_loss: 0.1443 - learning_rate: 0.0010
Epoch 2/50
470/470 ━━━━━━━━━━━━━━━━━━━━ 36s 78ms/step - accuracy: 0.6222 - dice_metric: 0.9243 - iou_metric: 0.8600 - loss: 0.1306 - val_accuracy: 0.6289 - val_dice_metric: 0.9358 - val_iou_metric: 0.8799 - val_loss: 0.1171 - learning_rate: 0.0010
Epoch 3/50
470/470 ━━━━━━━━━━━━━━━━━━━━ 39s 82ms/step - accuracy: 0.6257 - dice_metric: 0.9370 - iou_metric: 0.8819 - loss: 0.1089 - val_accuracy: 0.6281 - val_dice_metric: 0.9373 - val_iou_metric: 0.8826 - val_loss: 0.1148 - learning_rate: 0.0010
Epoch 4/50
470/470 ━━━━━━━━━━━━━━━━━━━━ 40s 85ms/step - accuracy: 0.6268 - dice_metric: 0.9412 - iou_metric: 0.8894 - loss: 0.1022 - val_accuracy: 0.6293 - val_dice_metric: 0.9436 - val_iou_metric: 0.8938 - val_loss: 0.0970 - learning_rate: 0.0010
Epoch 5